In [1]:
import numpy as np
import timeit
start = timeit.default_timer()
from datetime import datetime, date, timedelta
import pandas as pd
import yahoo_fin.stock_info as yf
import random
from sklearn.model_selection import train_test_split

In [2]:
nasdaqList = yf.tickers_nasdaq()
train, test = train_test_split(nasdaqList, test_size=0.3, random_state=0, shuffle=True)
trainSet1, trainSet2, trainSet3  = np.array_split(train,3)
trainSets = [trainSet1, trainSet2, trainSet3]

In [9]:
nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)
allList = []
nasdaqList = yf.tickers_nasdaq()
allList = set(nasdaqList + nyseList)

def baseTest(trainSet):
    now = datetime.now()
    now = now.strftime("%Y-%m-%d-%H:%M")
    transaction = pd.read_pickle("transactionTemplate.pkl")
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    for symbol in trainSet:
        try:
            df = pd.read_pickle("stockData/nyseNasdaq/"+symbol+".pkl")
            if df.empty:
                continue
        except:
            continue
        if not (df['close'] != 0).all():
            continue
        lastIndex = df.index[-1]
        driftOfLast = lastIndex.weekday()
        lastIndex = (lastIndex - timedelta(days=driftOfLast))
        if lastIndex != transaction.index[-1]:
            continue
        drift = -1
        for index, element in df.iterrows():
            if drift == -1:
                drift = index.weekday()
            index = index - timedelta(days= drift)
            index = index.strftime('%Y-%m-%d')
            rowNumber = transaction.index.get_loc(index)
            # 0 => 'holding'
            transaction.iat[rowNumber, 0].append((symbol,element.close,0))
    total = 100
    transaction.to_pickle('transactionDebug.pkl')
    prevIndex = ''
    first = True
    transactionCopy = transaction
    transactionCopy['total'] = 0
    for index, element in transactionCopy.iterrows():
        index = index.strftime('%Y-%m-%d')
        if len(element.holding) == 0:
            transactionCopy['total'][index] = total
            continue
        if first:
            first = False
            transactionCopy['total'][index] = total
        else:
            prevData = transactionCopy.loc[prevIndex].holding
            if len(prevData):
                total = 0
                close = 0
                for i in range(len(prevData)):
                    for j in range(len(element.holding)):
                        if element.holding[j][0] == prevData[i][0]:
                            close = element.holding[j][1]
                            break
                    total += close*prevData[i][2]
                transactionCopy['total'][index] = total
            else:
                transactionCopy['total'][index] = transactionCopy.loc[prevIndex].total
        if len(element.holding):
            allocation = total/len(element.holding)
        for i in range(len(element.holding)):
            element.holding[i] = (element.holding[i][0],element.holding[i][1],allocation/element.holding[i][1])
        prevIndex = index
    stockHolding  = 0
    for i in transactionCopy.iterrows():
        stockHolding += len(i[1]['holding'])
    print(stockHolding/(transactionCopy.shape[0]))
    print(transactionCopy.iloc[-1]['total'])
    resultFile = open("baseTest.txt","a")
    resultFile.write(str(stockHolding/(transactionCopy.shape[0]))+"\n")
    resultFile.write(str(transactionCopy.iloc[-1]['total'])+"\n")
    resultFile.close()
    return transactionCopy.iloc[-1]['total'],

In [10]:
print(baseTest(trainSets[1]))

/var/folders/n5/0wg686hs5jn0glfrk05fk_6r0000gn/T/ipykernel_5403/2316179473.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactionCopy['total'][index] = total
/opt/anaconda3/envs/titanic/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/var/folders/n5/0wg686hs5jn0glfrk05fk_6r0000gn/T/ipykernel_5403/2316179473.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

357.165131112686
429439.5574400954
(429439.5574400954,)
